In [1]:
import pandas as pd
import json
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import PowerTransformer
from imblearn.over_sampling import SMOTE
from itertools import groupby

def load_data(accounts_path, enquiry_path, flags_path):
    print('loading data')
    def my_json_file_reader(path):
        with open(path, 'r') as file:
            json_data = json.load(file)
        flattened_data = [item for sublist in json_data for item in sublist]
        return pd.json_normalize(flattened_data)

    accounts_df = my_json_file_reader(accounts_path)
    enquiry_df = my_json_file_reader(enquiry_path)
    flags_df = pd.read_csv(flags_path)
    
    return accounts_df, enquiry_df, flags_df

def preprocess_accounts(accounts_df):
    print('preprocessing acc data')
    accounts_df['open_date'] = pd.to_datetime(accounts_df['open_date'], errors='coerce')
    accounts_df['closed_date'] = pd.to_datetime(accounts_df['closed_date'], errors='coerce')

    accounts_df['payment_str_missing'] = accounts_df['payment_hist_string'].apply(
        lambda x: 1 if pd.isnull(x) or x == '' else 0
    )

    accounts_df['is_account_open'] = accounts_df['closed_date'].isna().astype(int)
    accounts_df['payment_hist_string'] = np.where(
        accounts_df['payment_str_missing'] == 1, 
        '000', 
        accounts_df['payment_hist_string']
    )
    accounts_df['Mob1'] = np.where(
    (accounts_df['payment_str_missing'] == 1)&(accounts_df['is_account_open']==0), 
    1, 
    0)
    
    return accounts_df

def preprocess_enquiries(enquiry_df, reference_date):
    print('preprocessing enq data')
    enquiry_df['enquiry_date'] = pd.to_datetime(enquiry_df['enquiry_date'], errors='coerce')
    enquiry_df['in_last_quarter'] = (enquiry_df['enquiry_date'] > (reference_date - pd.DateOffset(months=3)))
    enquiry_df['in_last_six_months'] = (enquiry_df['enquiry_date'] > (reference_date - pd.DateOffset(months=6)))
    enquiry_df['in_last_month'] = (enquiry_df['enquiry_date'] > (reference_date - pd.DateOffset(months=1)))
    
    return enquiry_df

def extract_payment_features(payment_hist_string):
    months = [payment_hist_string[i:i+3] for i in range(0, len(payment_hist_string), 3)]
    overdue_days = [int(month) for month in months]
    
    on_time_count = sum(1 for days in overdue_days if days == 0)
    max_days_overdue = max(overdue_days)
    average_days_overdue = sum(overdue_days) / len(overdue_days)
    months_late_count = sum(1 for days in overdue_days if days > 0)
    
    last_six_months = overdue_days[-6:] if len(overdue_days) >= 6 else overdue_days
    last_six_percent = sum(1 for days in last_six_months if days > 0)/6
    
    last_three_months = overdue_days[-3:] if len(overdue_days) >= 3 else overdue_days
    last_three_percent = sum(1 for days in last_three_months if days > 0)/3
    
    ever_0_by_3= 1 if sum(1 for days in last_three_months if days > 0)==0 else 0
    ever_1_by_3= 1 if sum(1 for days in last_three_months if days > 0)==1 else 0
    ever_2_by_3= 1 if sum(1 for days in last_three_months if days > 0)==2 else 0
    ever_3_by_3= 1 if sum(1 for days in last_three_months if days > 0)==3 else 0
    ever_30_plus = 1 if any(days > 30 for days in overdue_days) else 0
    longest_streak=max((len(g) for g in [list(g) for k, g in groupby(overdue_days) if k == 0]), default=0)
    
    return pd.Series([on_time_count, max_days_overdue, average_days_overdue, months_late_count,last_six_percent,last_three_percent,
                      
                      on_time_count+months_late_count,ever_0_by_3,ever_1_by_3,ever_2_by_3,ever_3_by_3,ever_30_plus,longest_streak])

def engineer_features(accounts_df, enquiry_df, reference_date):
    print('aggregation of data')
    tqdm.pandas(desc="Processing payment history")
    accounts_df[['on_time_count', 'max_days_overdue', 'average_days_overdue', 
                        'months_late_count','last_six_percent','last_three_percent','total_months',
                       'ever_0_by_3','ever_1_by_3','ever_2_by_3'
                 ,'ever_3_by_3','ever_30_plus','longest_streak']]= accounts_df['payment_hist_string'].progress_apply(extract_payment_features)
    print("pay hist str processing completed.")
    accounts_df['Never Del']= np.where(accounts_df['on_time_count'] == accounts_df['total_months'], 1, 0)

    open_accounts = accounts_df[accounts_df['is_account_open'] == 1]
    closed_accounts = accounts_df[accounts_df['is_account_open'] == 0]
    closed_accounts['loan_duration']=(closed_accounts['closed_date'] - closed_accounts['open_date']).dt.days

    print("open acc.")


    open_accounts_agg = open_accounts.groupby('uid').agg({
        'uid':['count'],
        'loan_amount': ['sum', 'mean', 'max'],  
        'amount_overdue': ['sum', 'mean'], 
        'on_time_count':['sum'],
        'months_late_count':['sum'],
        'max_days_overdue':['max'], 
        'average_days_overdue':['mean'],
        'last_six_percent':['max'],
        'last_three_percent':['max'],
        'total_months':['mean', 'max'],
            'ever_0_by_3':['sum'],
        'ever_1_by_3':['sum'],'ever_2_by_3':['sum'],'ever_3_by_3':['sum'],'ever_30_plus':['sum'],'longest_streak':['mean'],
        'Never Del':['sum'],'Mob1':['sum']
        
    }).reset_index()
    
    
    open_accounts_agg.columns = ['uid',
     'open_count',
     'open_total_loan_amount',
     'open_mean_loan_amount',
     'open_max_loan_amount',
     'open_total_amount_overdue',
     'open_mean_amount_overdue',
     'open_total_on_time_count',
     'open_total_late_count',
     'open_max_days_overdue',
     'open_average_days_overdue',
    'open_last_six_percent',
    'open_last_three_percent',
    'open_mean_loan_duration',
    'open_max_loan_duration',
         'open_ever_0_by_3','open_ever_1_by_3','open_ever_2_by_3','open_ever_3_by_3','open_ever_30_plus','open_longest_streak',
    'open_Never_del','open_Mob1']
                           
    print("closed acc.")
                        
                                
    
    closed_accounts_agg = closed_accounts.groupby('uid').agg({
        'uid':['count'],
        'loan_amount': ['sum', 'mean', 'max'], 
        'amount_overdue': ['sum', 'mean'],  
        'on_time_count':['sum'],
        'months_late_count':['sum'],
        'max_days_overdue':['max'], 
        'average_days_overdue':['mean'],
        'loan_duration':['mean', 'max'],
            'ever_0_by_3':['sum'],
        'ever_1_by_3':['sum'],'ever_2_by_3':['sum'],'ever_3_by_3':['sum'],'ever_30_plus':['sum'],'longest_streak':['mean'],
        'Never Del':['sum'],'Mob1':['sum']
    }).reset_index()
    
    closed_accounts_agg.columns = ['uid',
    'closed_count',
     'closed_total_loan_amount',
     'closed_mean_loan_amount',
     'closed_max_loan_amount',
     'closed_total_amount_overdue',
     'closed_mean_amount_overdue',
     'closed_total_on_time_count',
     'closed_total_late_count',
     'closed_max_days_overdue',
     'closed_average_days_overdue',
    'closed_mean_loan_duration',
    'closed_max_loan_duration',
         'closed_ever_0_by_3','closed_ever_1_by_3','closed_ever_2_by_3','closed_ever_3_by_3','closed_ever_30_plus','closed_longest_streak',
    'closed_Never_del','closed_Mob1'
    
                                ]
    
    open_accounts_agg['open_on_time_percent']=open_accounts_agg['open_total_on_time_count']/(open_accounts_agg['open_total_on_time_count']+open_accounts_agg['open_total_late_count'])
    closed_accounts_agg['closed_on_time_percent']=closed_accounts_agg['closed_total_on_time_count']/(closed_accounts_agg['closed_total_on_time_count']+closed_accounts_agg['closed_total_late_count'])
    accounts_agg = open_accounts_agg.merge(closed_accounts_agg, on='uid', how='outer')

    print("grouping enq")

    enquiry_agg = enquiry_df.groupby('uid').agg({
    'enquiry_amt': ['sum', 'mean', 'max', 'count'],  
    'enquiry_type': ['nunique'],  
        'in_last_quarter': ['sum'],  
    'in_last_six_months': ['sum'],
    'in_last_month':['sum']
    }).reset_index()

    enquiry_agg.columns = [
        'uid', 
        'enquiry_amt_sum', 
        'enquiry_amt_mean', 
        'enquiry_amt_max', 
        'enquiry_count', 
        'unique_enquiry_types', 
        'last_quarter_enquiries', 
        'last_six_months_enquiries',
        'last_month_enquiries'
    ]

    return accounts_agg, enquiry_agg

def calculate_reference_date(accounts_df):
    #I believe I need a reference date against which I can check the enquiry date and such features
    #I think this is old data and using today's date will be wrong
    #therefore I am going to use the below date which is calculated using the open date and length of the payment history string
    print('calculating ref date')
    random_sample_df = accounts_df[accounts_df['is_account_open'] == 1].sample(n=5000, random_state=1)
    random_sample_df['count_months'] = random_sample_df['payment_hist_string'].str.len() / 3
    random_sample_df['ref_date'] = random_sample_df.apply(
        lambda row: row['open_date'] + pd.DateOffset(months=row['count_months']),
        axis=1
    )
    reference_date = max(random_sample_df['ref_date'])
    return reference_date

def train_pipeline(accounts_path, enquiry_path, flags_path):
    
    accounts_df, enquiry_df, flags_df = load_data(accounts_path, enquiry_path, flags_path)
    accounts_df = preprocess_accounts(accounts_df)
    
    reference_date = calculate_reference_date(accounts_df)
    enquiry_df = preprocess_enquiries(enquiry_df, reference_date)
    
    accounts_agg, enquiry_agg = engineer_features(accounts_df, enquiry_df, reference_date)
    
    training_df = flags_df.merge(accounts_agg, on='uid', how='left')
    training_df = training_df.merge(enquiry_agg, on='uid', how='left')


    encoder = OneHotEncoder(sparse=False, drop='first') 
    encoded_features = encoder.fit_transform(training_df[['NAME_CONTRACT_TYPE']])
    encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['NAME_CONTRACT_TYPE']))
    
    df_encoded = pd.concat([training_df, encoded_df], axis=1)
    
    df_encoded.drop(columns=['NAME_CONTRACT_TYPE'], inplace=True)
    
    df_encoded['no_closed_loans'] = np.where(df_encoded['closed_count'].isna(), 1, 0)
    
    df_encoded['no_open_loans'] = np.where(df_encoded['open_count'].isna(), 1, 0)
    
    df_encoded.fillna(0.0001, inplace=True)

    
    X = df_encoded.drop(columns=['TARGET', 'uid'])
    y = df_encoded['TARGET']

    print('handling outliers')
    for col in X.columns:
        Q1 = X[col].quantile(0.25)
        Q3 = X[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        X[col] = np.where(X[col] > upper_bound, upper_bound, X[col])
        X[col] = np.where(X[col] < lower_bound, lower_bound, X[col])
        X[col]=np.cbrt(X[col])

    

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
    
    X_train, X_val, y_train, y_val = train_test_split(X_scaled_df, y, test_size=0.2, random_state=42, stratify=y)
    
    model = LGBMClassifier(random_state=42, scale_pos_weight=len(y_train) / sum(y_train))
    model.fit(X_train, y_train)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    roc_auc = roc_auc_score(y_val, y_pred)
    print(f"Training ROC AUC: {roc_auc}")
    
    return model, scaler,encoder, reference_date  


def test_pipeline(accounts_path, enquiry_path, flags_path, model, scaler, encoder, reference_date):
    accounts_df, enquiry_df, flags_df = load_data(accounts_path, enquiry_path, flags_path)
    accounts_df = preprocess_accounts(accounts_df)
    
    enquiry_df = preprocess_enquiries(enquiry_df, reference_date)

    accounts_agg, enquiry_agg = engineer_features(accounts_df, enquiry_df, reference_date)
    
    test_df = flags_df.merge(accounts_agg, on='uid', how='left')
    test_df = test_df.merge(enquiry_agg, on='uid', how='left')

    encoded_features = encoder.transform(test_df[['NAME_CONTRACT_TYPE']])
    encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['NAME_CONTRACT_TYPE']))
    
    df_encoded = pd.concat([test_df, encoded_df], axis=1)

    df_encoded.drop(columns=['NAME_CONTRACT_TYPE'], inplace=True)

    df_encoded['no_closed_loans'] = np.where(df_encoded['closed_count'].isna(), 1, 0)
    df_encoded['no_open_loans'] = np.where(df_encoded['open_count'].isna(), 1, 0)

    df_encoded.fillna(0.0001, inplace=True)

    X_test = df_encoded.drop(columns=['uid'])

    for col in X_test.columns:
        Q1 = X_test[col].quantile(0.25)
        Q3 = X_test[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        X_test[col] = np.where(X_test[col] > upper_bound, upper_bound, X_test[col])
        X_test[col] = np.where(X_test[col] < lower_bound, lower_bound, X_test[col])
        X_test[col] = np.cbrt(X_test[col])

    X_test_scaled = scaler.transform(X_test)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

    y_test_pred = model.predict_proba(X_test_scaled_df)[:, 1]

    test_df['pred'] = y_test_pred
    return test_df[['uid','pred']]




In [2]:

model, scaler,encoder,reference_date = train_pipeline('data/train/accounts_data_train.json', 
                               'data/train/enquiry_data_train.json', 
                               'data/train/train_flag.csv')

loading data
preprocessing acc data
calculating ref date
preprocessing enq data
aggregation of data


Processing payment history: 100%|██████████████████████████████████████████| 1245310/1245310 [12:12<00:00, 1699.22it/s]


pay hist str processing completed.


C:\Users\CLD\AppData\Local\Temp\ipykernel_29412\1234274756.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closed_accounts['loan_duration']=(closed_accounts['closed_date'] - closed_accounts['open_date']).dt.days


open acc.
closed acc.
grouping enq


C:\Users\CLD\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


handling outliers
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 16846, number of negative: 192260
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.161619 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4658
[LightGBM] [Info] Number of data points in the train set: 209106, number of used features: 53
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080562 -> initscore=-2.434735
[LightGBM] [Info] Start training from score -2.434735
Training ROC AUC: 0.6416741326042006


In [3]:
test_results = test_pipeline('data/test/accounts_data_test.json', 
                             'data/test/enquiry_data_test.json', 
                             'data/test/test_flag.csv', 
                             model, 
                             scaler,encoder,reference_date)


test_results.to_csv('final_submission_clyde_dcosta.csv', index=False)



loading data
preprocessing acc data
preprocessing enq data
aggregation of data


Processing payment history: 100%|████████████████████████████████████████████| 220013/220013 [00:54<00:00, 4062.68it/s]
C:\Users\CLD\AppData\Local\Temp\ipykernel_29412\1234274756.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closed_accounts['loan_duration']=(closed_accounts['closed_date'] - closed_accounts['open_date']).dt.days


pay hist str processing completed.
open acc.
closed acc.
grouping enq


In [4]:
test_results.head()


,uid,pred
0,CMO22835242,0.504667
1,MRJ34316727,0.490263
2,UAV00534378,0.604203
3,IPQ08190402,0.430404
4,NQN84331006,0.493659


In [6]:
"""
NOTE:
Few of the things I tried which are not included in this final code:
1. SMOTE: I used this for oversampling of the minority class expecting this to lead to improvement but somehow roc_auc score decreased by ~9 percent.
2. using this:

 def grouping_loans(loan):
    if loan in ['Credit card','Revolving loans','Interbank credit']:
        return 'Cash loans'
    else:
        return 'Revolving loans'
        
I grouped the loan_types under 2 categories as seen in the flag dataframe. 
I figured that grouping by and then using the specific loan_type as the instance would lead to more relevant information being consumed 
and hence better predictions (I do this at my current job and it works well). 
Therefore I had initially grouped by uid and NAME_CONTRACT_TYPE; but this again led to a decrease in accuracy.

3.for feature selection I used the following code snippets for quick selection. I usually use RFE and save all the scores in an excel sheet along with the used features and then choose suitable sets. Due to lack of time, I have directly used this here.

###for tree based:

importances = model.feature_importances_

# Create a DataFrame for feature importances
feature_importance_df = pd.DataFrame({'feature': X.columns, 'importance': importances})

# Sort by importance
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

###for log reg

coefficients = model.coef_[0] 
feature_names = X.columns

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': coefficients
})

feature_importance_df['absolute_importance'] = feature_importance_df['importance'].abs()
feature_importance_df['percent']=feature_importance_df['absolute_importance']*100/feature_importance_df['absolute_importance'].sum()
feature_importance_df = feature_importance_df.sort_values(by='percent', ascending=False)


"""

"\nNOTE:\nFew of the things I tried which are not included in this final code:\n1. SMOTE: I used this for oversampling of the minority class expecting this to lead to improvement but somehow roc_auc score decreased by ~9 percent.\n2. using this:\n\n def grouping_loans(loan):\n    if loan in ['Credit card','Revolving loans','Interbank credit']:\n        return 'Cash loans'\n    else:\n        return 'Revolving loans'\n        \nI grouped the loan_types under 2 categories as seen in the flag dataframe. \nI figured that grouping by and then using the specific loan_type as the instance would lead to more relevant information being consumed \nand hence better predictions (I do this at my current job and it works well). \nTherefore I had initially grouped by uid and NAME_CONTRACT_TYPE; but this again led to a decrease in accuracy.\n\n3.for feature selection I used the following code snippets for quick selection. I usually use RFE and save all the scores in an excel sheet along with the used 